In [632]:
from tkinter import YView, XView

import numpy as np
import pandas as pd
np.random.seed = 123
data = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [633]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [634]:
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Cabin'].fillna('N',inplace=True)
data['Embarked'].fillna('N',inplace=True)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].mean(),inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3057721139.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [635]:
data.loc[data["Sex"] == "male", "Sex_encode"] = 0
data.loc[data["Sex"] == "female", "Sex_encode"] = 1

In [636]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행.
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [637]:
Y = data['Survived']
X = data.drop('Survived', axis=1)


In [638]:
X = transform_features(X)

/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)
/var/folders/9d/lcchyqrx6jq_hz6pm4d1j7z80000gn/T/ipykernel_58162/3807705002.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

In [639]:
import numpy as np

# 로그 변환 적용 (값이 0 이하인 경우를 방지하기 위해 작은 값 더하기)
X['Fare'] = np.log(X['Fare'] + 1)
X['Age'] = np.log(X['Age'] + 1)



In [640]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, Y, \
                                                  test_size=0.2, random_state=11)

In [641]:
def batch_sampler(data, target, batch_size):
    tot = len(data)
    total_iter = tot // batch_size
    for i in range(total_iter):
        yield (data[i * batch_size:(i + 1) * batch_size],
               target[i * batch_size:(i + 1) * batch_size])


sampler = batch_sampler(data=X_train, target=y_train, batch_size=32)
L = len(X_train) // 32
for i in range(L):
    next(sampler)
print(X_train)

     Pclass  Sex       Age  SibSp  Parch      Fare  Cabin  Embarked  \
333       3    1  2.833213      2      0  2.944439      7         3   
662       1    1  3.871201      0      0  3.280441      4         3   
382       3    1  3.496508      0      0  2.188856      7         3   
331       1    1  3.839452      0      0  3.384390      2         3   
149       2    1  3.761200      0      0  2.639057      7         3   
..      ...  ...       ...    ...    ...       ...    ...       ...   
269       1    0  3.583519      0      0  4.917301      2         3   
337       1    0  3.737670      0      0  4.908972      4         0   
91        3    1  3.044522      0      0  2.180892      7         3   
80        3    1  3.135494      0      0  2.302585      7         3   
703       3    1  3.258097      0      0  2.168105      7         2   

     Sex_encode  
333         0.0  
662         0.0  
382         0.0  
331         0.0  
149         0.0  
..          ...  
269         1.0  
337

In [642]:
import numpy as np
from tqdm import tqdm
class LogisticRegression:
    def __init__(self, learning_rate=0.01, epochs=400,batch_size = 32, n_features=9):
        # 초기 가중치를 무작위로 설정
        self.weights = np.random.randn(n_features)
        self.b = np.random.randn()
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.loss_history = []
        self.batch_size = batch_size

    def sigmoid(self, z):
        """Sigmoid 활성화 함수"""
        return 1.0 / (1 + np.exp(-z))

    def compute_loss(self, preds, target):
        """교차 엔트로피 손실 계산"""
        # 로그의 안정성을 위해 값들을 클리핑
        preds = np.clip(preds, 1e-9, 1 - 1e-9)  # 0과 1의 범위를 벗어나지 않도록 함
        loss = -((target * np.log(preds)) + (1 - target) * np.log(1 - preds)).mean()
        return loss


    def fit(self, X, y, batch_sampler):
        """모델 훈련 함수"""
        for epoch in tqdm(range(self.epochs)):
            sampler = batch_sampler(data=X, target=y, batch_size=self.batch_size)
            batch_loss = []
    
            for X_batch, Y_batch in sampler:
                # 순방향 전파
                preds = self.sigmoid(np.dot(X_batch, self.weights) + self.b)
    
                # 손실 계산
                loss = self.compute_loss(preds, Y_batch)
                batch_loss.append(loss)
    
                # 그래디언트 계산
                dloss_dz = preds - Y_batch
                weights_deriv = np.dot(X_batch.T, dloss_dz) / X_batch.shape[0]
                b_deriv = dloss_dz.mean()
    
                # 가중치 및 바이어스 업데이트
                self.weights -= self.learning_rate * weights_deriv
                self.b -= self.learning_rate * b_deriv
    
            # Epoch별 손실 저장
            epoch_loss = np.mean(batch_loss)
            self.loss_history.append(epoch_loss)
            
            # 에포크마다 손실 출력
            print(f"Epoch {epoch + 1}/{self.epochs} - Loss: {epoch_loss:.4f}")


    def predict(self, X):
        """모델 예측 함수"""
        linear_comb = np.dot(X, self.weights) + self.b
        return self.sigmoid(linear_comb) > 0.55

    def accuracy(self, X, y):
        """정확도 계산 함수"""
        preds = self.predict(X)
        return (preds == y).mean() * 100




In [643]:
model = LogisticRegression(learning_rate=0.01, epochs=200,batch_size=32, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


 15%|█▌        | 30/200 [00:00<00:01, 141.11it/s]

Epoch 1/200 - Loss: 1.1720
Epoch 2/200 - Loss: 1.1186
Epoch 3/200 - Loss: 1.0902
Epoch 4/200 - Loss: 1.0633
Epoch 5/200 - Loss: 1.0376
Epoch 6/200 - Loss: 1.0129
Epoch 7/200 - Loss: 0.9891
Epoch 8/200 - Loss: 0.9664
Epoch 9/200 - Loss: 0.9446
Epoch 10/200 - Loss: 0.9237
Epoch 11/200 - Loss: 0.9036
Epoch 12/200 - Loss: 0.8844
Epoch 13/200 - Loss: 0.8661
Epoch 14/200 - Loss: 0.8486
Epoch 15/200 - Loss: 0.8319
Epoch 16/200 - Loss: 0.8160
Epoch 17/200 - Loss: 0.8009
Epoch 18/200 - Loss: 0.7866
Epoch 19/200 - Loss: 0.7731
Epoch 20/200 - Loss: 0.7604
Epoch 21/200 - Loss: 0.7485
Epoch 22/200 - Loss: 0.7372
Epoch 23/200 - Loss: 0.7267
Epoch 24/200 - Loss: 0.7168
Epoch 25/200 - Loss: 0.7076
Epoch 26/200 - Loss: 0.6990
Epoch 27/200 - Loss: 0.6909
Epoch 28/200 - Loss: 0.6834
Epoch 29/200 - Loss: 0.6763
Epoch 30/200 - Loss: 0.6697


 22%|██▎       | 45/200 [00:00<00:01, 140.14it/s]

Epoch 31/200 - Loss: 0.6635
Epoch 32/200 - Loss: 0.6576
Epoch 33/200 - Loss: 0.6521
Epoch 34/200 - Loss: 0.6469
Epoch 35/200 - Loss: 0.6420
Epoch 36/200 - Loss: 0.6373
Epoch 37/200 - Loss: 0.6329
Epoch 38/200 - Loss: 0.6286
Epoch 39/200 - Loss: 0.6245
Epoch 40/200 - Loss: 0.6206
Epoch 41/200 - Loss: 0.6169
Epoch 42/200 - Loss: 0.6133
Epoch 43/200 - Loss: 0.6098
Epoch 44/200 - Loss: 0.6064
Epoch 45/200 - Loss: 0.6031


 30%|███       | 60/200 [00:00<00:01, 139.77it/s]

...

Epoch 136/200 - Loss: 0.4729
Epoch 137/200 - Loss: 0.4724
Epoch 138/200 - Loss: 0.4720
Epoch 139/200 - Loss: 0.4715
Epoch 140/200 - Loss: 0.4711
Epoch 141/200 - Loss: 0.4707
Epoch 142/200 - Loss: 0.4703
Epoch 143/200 - Loss: 0.4699
Epoch 144/200 - Loss: 0.4695
Epoch 145/200 - Loss: 0.4691
Epoch 146/200 - Loss: 0.4687
Epoch 147/200 - Loss: 0.4684
Epoch 148/200 - Loss: 0.4680
Epoch 149/200 - Loss: 0.4677
Epoch 150/200 - Loss: 0.4673
Epoch 151/200 - Loss: 0.4670
Epoch 152/200 - Loss: 0.4667
Epoch 153/200 - Loss: 0.4664
Epoch 154/200 - Loss: 0.4661
Epoch 155/200 - Loss: 0.4658
Epoch 156/200 - Loss: 0.4655
Epoch 157/200 - Loss: 0.4652
Epoch 158/200 - Loss: 0.4649
Epoch 159/200 - Loss: 0.4646
Epoch 160/200 - Loss: 0.4644
Epoch 161/200 - Loss: 0.4641
Epoch 162/200 - Loss: 0.4639
Epoch 163/200 - Loss: 0.4636
Epoch 164/200 - Loss: 0.4634
Epoch 165/200 - Loss: 0.4631


 90%|████████▉ | 179/200 [00:01<00:00, 138.80it/s]

Epoch 166/200 - Loss: 0.4629
Epoch 167/200 - Loss: 0.4627
Epoch 168/200 - Loss: 0.4625
Epoch 169/200 - Loss: 0.4623
Epoch 170/200 - Loss: 0.4621
Epoch 171/200 - Loss: 0.4619
Epoch 172/200 - Loss: 0.4617
Epoch 173/200 - Loss: 0.4615
Epoch 174/200 - Loss: 0.4613
Epoch 175/200 - Loss: 0.4611
Epoch 176/200 - Loss: 0.4609
Epoch 177/200 - Loss: 0.4608
Epoch 178/200 - Loss: 0.4606
Epoch 179/200 - Loss: 0.4604


100%|██████████| 200/200 [00:01<00:00, 137.86it/s]

Epoch 180/200 - Loss: 0.4603
Epoch 181/200 - Loss: 0.4601
Epoch 182/200 - Loss: 0.4600
Epoch 183/200 - Loss: 0.4598
Epoch 184/200 - Loss: 0.4597
Epoch 185/200 - Loss: 0.4595
Epoch 186/200 - Loss: 0.4594
Epoch 187/200 - Loss: 0.4593
Epoch 188/200 - Loss: 0.4591
Epoch 189/200 - Loss: 0.4590
Epoch 190/200 - Loss: 0.4589
Epoch 191/200 - Loss: 0.4588
Epoch 192/200 - Loss: 0.4586
Epoch 193/200 - Loss: 0.4585
Epoch 194/200 - Loss: 0.4584
Epoch 195/200 - Loss: 0.4583
Epoch 196/200 - Loss: 0.4582
Epoch 197/200 - Loss: 0.4581
Epoch 198/200 - Loss: 0.4580
Epoch 199/200 - Loss: 0.4579
Epoch 200/200 - Loss: 0.4578
Test Accuracy: 84.36%


In [644]:
model = LogisticRegression(learning_rate=0.01, epochs=200,batch_size=64, n_features=9)
model.fit(X_train, y_train, batch_sampler=batch_sampler)
# X_test에 대한 예측값 생성
predictions = model.predict(X_test)

# 정확도 계산
def calculate_accuracy(predictions, y_test):
    """예측값과 실제값을 비교하여 정확도를 계산"""
    correct_predictions = (predictions == y_test).sum()
    accuracy = (correct_predictions / len(y_test)) * 100
    return accuracy

# y_test와 비교하여 정확도 계산
accuracy = calculate_accuracy(predictions, y_test)
print(f"Test Accuracy: {accuracy:.2f}%")


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/200 - Loss: 4.5354
Epoch 2/200 - Loss: 3.5806
Epoch 3/200 - Loss: 2.6985
Epoch 4/200 - Loss: 1.9789
Epoch 5/200 - Loss: 1.4982
Epoch 6/200 - Loss: 1.2455
Epoch 7/200 - Loss: 1.1482
Epoch 8/200 - Loss: 1.1144
Epoch 9/200 - Loss: 1.0961
Epoch 10/200 - Loss: 1.0806
Epoch 11/200 - Loss: 1.0655
Epoch 12/200 - Loss: 1.0507
Epoch 13/200 - Loss: 1.0361
Epoch 14/200 - Loss: 1.0217
Epoch 15/200 - Loss: 1.0074
Epoch 16/200 - Loss: 0.9934
Epoch 17/200 - Loss: 0.9795
Epoch 18/200 - Loss: 0.9659
Epoch 19/200 - Loss: 0.9524
Epoch 20/200 - Loss: 0.9392
Epoch 21/200 - Loss: 0.9262


 14%|█▍        | 29/200 [00:00<00:00, 287.22it/s]

Epoch 22/200 - Loss: 0.9134
Epoch 23/200 - Loss: 0.9009
Epoch 24/200 - Loss: 0.8885
Epoch 25/200 - Loss: 0.8765
Epoch 26/200 - Loss: 0.8646
Epoch 27/200 - Loss: 0.8531
Epoch 28/200 - Loss: 0.8417
Epoch 29/200 - Loss: 0.8307
Epoch 30/200 - Loss: 0.8199
Epoch 31/200 - Loss: 0.8094
Epoch 32/200 - Loss: 0.7991
Epoch 33/200 - Loss: 0.7891
Epoch 34/200 - Loss: 0.7794
Epoch 35/200 - Loss: 0.7700
Epoch 36/200 - Loss: 0.7608
Epoch 37/200 - Loss: 0.7519
Epoch 38/200 - Loss: 0.7433
Epoch 39/200 - Loss: 0.7350
Epoch 40/200 - Loss: 0.7270
Epoch 41/200 - Loss: 0.7192
Epoch 42/200 - Loss: 0.7117
Epoch 43/200 - Loss: 0.7044
Epoch 44/200 - Loss: 0.6974
Epoch 45/200 - Loss: 0.6907
Epoch 46/200 - Loss: 0.6842
Epoch 47/200 - Loss: 0.6779
Epoch 48/200 - Loss: 0.6719
Epoch 49/200 - Loss: 0.6661
Epoch 50/200 - Loss: 0.6605
Epoch 51/200 - Loss: 0.6551
Epoch 52/200 - Loss: 0.6499
Epoch 53/200 - Loss: 0.6449
Epoch 54/200 - Loss: 0.6401
Epoch 55/200 - Loss: 0.6355
Epoch 56/200 - Loss: 0.6310
Epoch 57/200 - Loss:

 44%|████▍     | 89/200 [00:00<00:00, 292.18it/s]

...

Epoch 90/200 - Loss: 0.5430
Epoch 91/200 - Loss: 0.5415
Epoch 92/200 - Loss: 0.5400
Epoch 93/200 - Loss: 0.5386
Epoch 94/200 - Loss: 0.5373
Epoch 95/200 - Loss: 0.5359
Epoch 96/200 - Loss: 0.5346
Epoch 97/200 - Loss: 0.5334
Epoch 98/200 - Loss: 0.5321
Epoch 99/200 - Loss: 0.5309
Epoch 100/200 - Loss: 0.5297
Epoch 101/200 - Loss: 0.5286
Epoch 102/200 - Loss: 0.5274
Epoch 103/200 - Loss: 0.5263
Epoch 104/200 - Loss: 0.5252
Epoch 105/200 - Loss: 0.5242
Epoch 106/200 - Loss: 0.5231
Epoch 107/200 - Loss: 0.5221
Epoch 108/200 - Loss: 0.5211
Epoch 109/200 - Loss: 0.5202
Epoch 110/200 - Loss: 0.5192
Epoch 111/200 - Loss: 0.5183
Epoch 112/200 - Loss: 0.5174
Epoch 113/200 - Loss: 0.5165
Epoch 114/200 - Loss: 0.5156
Epoch 115/200 - Loss: 0.5147
Epoch 116/200 - Loss: 0.5139
Epoch 117/200 - Loss: 0.5131
Epoch 118/200 - Loss: 0.5122
Epoch 119/200 - Loss: 0.5114
Epoch 120/200 - Loss: 0.5107
Epoch 121/200 - Loss: 0.5099
Epoch 122/200 - Loss: 0.5092
Epoch 123/200 - Loss: 0.5084
Epoch 124/200 - Loss: 0.

 90%|████████▉ | 179/200 [00:00<00:00, 288.93it/s]

Epoch 150/200 - Loss: 0.4928
Epoch 151/200 - Loss: 0.4924
Epoch 152/200 - Loss: 0.4919
Epoch 153/200 - Loss: 0.4915
Epoch 154/200 - Loss: 0.4910
Epoch 155/200 - Loss: 0.4906
Epoch 156/200 - Loss: 0.4902
Epoch 157/200 - Loss: 0.4898
Epoch 158/200 - Loss: 0.4894
Epoch 159/200 - Loss: 0.4890
Epoch 160/200 - Loss: 0.4886
Epoch 161/200 - Loss: 0.4882
Epoch 162/200 - Loss: 0.4878
Epoch 163/200 - Loss: 0.4875
Epoch 164/200 - Loss: 0.4871
Epoch 165/200 - Loss: 0.4867
Epoch 166/200 - Loss: 0.4864
Epoch 167/200 - Loss: 0.4860
Epoch 168/200 - Loss: 0.4857
Epoch 169/200 - Loss: 0.4854
Epoch 170/200 - Loss: 0.4850
Epoch 171/200 - Loss: 0.4847
Epoch 172/200 - Loss: 0.4844
Epoch 173/200 - Loss: 0.4840
Epoch 174/200 - Loss: 0.4837
Epoch 175/200 - Loss: 0.4834
Epoch 176/200 - Loss: 0.4831
Epoch 177/200 - Loss: 0.4828
Epoch 178/200 - Loss: 0.4825
Epoch 179/200 - Loss: 0.4822


100%|██████████| 200/200 [00:00<00:00, 289.74it/s]

Epoch 180/200 - Loss: 0.4819
Epoch 181/200 - Loss: 0.4817
Epoch 182/200 - Loss: 0.4814
Epoch 183/200 - Loss: 0.4811
Epoch 184/200 - Loss: 0.4808
Epoch 185/200 - Loss: 0.4806
Epoch 186/200 - Loss: 0.4803
Epoch 187/200 - Loss: 0.4800
Epoch 188/200 - Loss: 0.4798
Epoch 189/200 - Loss: 0.4795
Epoch 190/200 - Loss: 0.4793
Epoch 191/200 - Loss: 0.4790
Epoch 192/200 - Loss: 0.4788
Epoch 193/200 - Loss: 0.4786
Epoch 194/200 - Loss: 0.4783
Epoch 195/200 - Loss: 0.4781
Epoch 196/200 - Loss: 0.4779
Epoch 197/200 - Loss: 0.4776
Epoch 198/200 - Loss: 0.4774
Epoch 199/200 - Loss: 0.4772
Epoch 200/200 - Loss: 0.4770
Test Accuracy: 83.24%
